<h3>Rebuild ENEM's answers<br></h3>

The enem's test with 45 single choice math questions, followed by alternatives ranging from A to E. In this scenario the last five answers have been removed from the test dataset, so you will rebuild them from the final average result - creating a model to predict the marked down answers.

In [1]:
import pandas as pd
import numpy as np

from src.send_answer import send_answer
from src.models.mode_prediction import predict_answers
from src.models.score import score, naive_approach

pd.set_option('display.max_columns', 500)

Considering the available options to choose from (A to E) and including the possibility to leave the question blank (*) the student last 5 answers would have a 1/7776 probability in a uniform distribution.

### Strategy
The strategy to overcome the dataset diversity consists on segmenting data in subsets in which the likelihood of similar answers gets increased. Not only segmenting the dataset by the types of tests employed, but by using the model defined in the previous challenge to recreate the math grades, we may better segment the dataset

In [2]:
# input data
train = pd.read_csv('../../data/processed/train3.csv').set_index('NU_INSCRICAO')
validation = pd.read_csv('../../data/processed/validation3.csv').set_index('NU_INSCRICAO')
test = pd.read_csv('../../data/processed/test3.csv').set_index('NU_INSCRICAO')

<h3>Estimate</h3><br>
The underlying idea of the following function is to predict the most common written answer for the segmented performance quartile as well as for its corresponding test

In [3]:
# number of answers to predict per row
streak = 5
train_answers = predict_answers(train, train, streak)
validation_answers = predict_answers(train, validation, streak)
test_answers = predict_answers(train, test, streak)

### Results

In [4]:
print('Naive approach accuracy: {:.2f}%'.format(naive_approach(train)*100))
print('Traning set accuracy: {:.2f}%'.format(score(train.TX_RESPOSTAS_MT.str[-streak:], train_answers)*100))
print('Validation set accuracy: {:.2f}%'.format(score(validation.TX_RESPOSTAS_MT.str[-streak:], validation_answers)*100))

Naive approach accuracy: 19.96%
Traning set accuracy: 32.27%
Validation set accuracy: 32.16%


In [7]:
# send answers
answer = test_answers.copy().to_frame()
answer = answer.rename(index=str, columns={"PREDICTION": "TX_RESPOSTAS_MT"})
#send_answer(answer.reset_index(), 3)